In [287]:
import pandas as pd
pd.set_option("display.max_colwidth", None)
toxic_comments = pd.read_csv("/Users/ekaterinakastaleva/russian_comments_from_2ch_pikabu.csv")

In [288]:
import numpy as np
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ekaterinakastaleva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [289]:
toxic_comments

,comment,toxic,translated
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"Camels, for what? Morons, bl ..."
1,"Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n",1.0,"Ukrainians, this is an outlet vent zatyukanogo Russians, they say, out, and Ukrainians worse. If there were no Ukrainians, jelly would have invented them."
2,Собаке - собачья смерть\n,1.0,Dog - Dog Death
3,"Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n",1.0,"Refresh the page, moron. This is also not an insult, but a proven fact - I will not write a non-moron to myself in the plural. Or do we believe in you - are you and your imaginary friends?"
4,"тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n",1.0,did not convince you of the 6-page pdf that Skripale was poisoned by Russia? Are you trying to analyze and think? Padded jacket or what?)
...,...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и сторонник демократии и свободы слова закукарекал.\n,1.0,Smelly soviet cattle came running and aching. And here the supporter of democracy and freedom of speech crowed.
14408,"А кого любить? Гоблина тупорылого что-ли? Или какую-нибудь продажную суку из демшизы? Нет уж, спасибо не надо.\n",1.0,"And whom to love? Goblin blunt or what? Or some corrupt demshiz bitch? No, thanks, no."
14409,"Посмотрел Утомленных солнцем 2. И оказалось, что это хороший фильм, такая высокобюджетная артхаусятина, к которой могут быть претензии только потому, что спиздили-распилили и вообще ТАК НЕ БЫВАЕТ. Ну нахуй этих критиков. Обзоры длиннее фильмов, петросянство хуже рашкокомедий, ебанутая ненависть и доебки по мелочам.\n",0.0,"I watched Burnt by the Sun 2. And it turned out that this is a good film, such a high-budget arthouse, to which there can be a complaint only because it was spit-sawed and in general SO DO NOT HAPPEN. Well fuck these critics. Reviews are longer than films, Petrosianism is worse than rash comedies, fucked hatred and trivialities."
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ НЕТ ОБСУЖДЕНИЯ ПОЛИТИКИ. СВОБОДНОЕ ОБЩЕНИЕ ЭТО В b. ЭТО ТОЖЕ САМОЕ ЕСЛИ Я НА ДОСКЕ О ПОЛИТИКЕ СОЗДАМ ТРЕД О ТОМ КАК ПЛОХО БЕЗ ТЯНОЧКИ. ТАК ЧТО УЕБЫВАЙТЕ В Б ИЛИ НВР СО СВОИМ ЧАТИКОМ ПРЕСТАРЕЛЫХ ГОМОСЕКОВ!\n,1.0,CRIMEA THREAD VIOLATES THE RULES OF SECTION. TK THERE IS NO POLICY DISCUSSION IN IT. FREE COMMUNICATION THIS IN b. THIS IS ALSO THE MOST IF I AM ON THE POLICY BOARD CREATING THREADS ABOUT BAD WITHOUT A TANCH. SO VISIT TO B or NVR WITH YOUR CHATIK OF THE ELDER GOMOSEKS!


In [290]:
toxic_comments.columns=[ "comment", "toxic","translated"]

In [291]:
#I dont need the translated-column, because of the bad translation
toxic_comments=toxic_comments[["comment","toxic"]]
toxic_comments.columns

Index(['comment', 'toxic'], dtype='object')

In [292]:
only_toxic_comments = toxic_comments[toxic_comments["toxic"]==1.0]
normal_comments = toxic_comments[toxic_comments["toxic"]==0.0]
#I have now two classes: normal_comments and only_toxic_comments
normal_comments = normal_comments.iloc[:int(4800)]
print("There are " + str(len(normal_comments)) + " comments with non toxic meaning.")
only_toxic_comments = only_toxic_comments.iloc[:int(4800)]
print("There are " + str(len(only_toxic_comments)) + " comments with toxic meaning.")
toxic_dataset = pd.concat([only_toxic_comments, normal_comments])
print(len(toxic_dataset))

There are 4800 comments with non toxic meaning.
There are 4800 comments with toxic meaning.
9600


In [293]:
toxic_dataset=toxic_dataset.comment

In [294]:
#lowercase all words
toxic_dataset = toxic_dataset.str.lower()

In [295]:
type(toxic_comments)

pandas.core.frame.DataFrame

In [296]:
print(toxic_dataset[5:10])

5                                     для каких стан является эталоном современная система здравоохранения рф? для зимбабве? ты тупой? хохлы\n
7     упад т! там нельзя строить! технологий нет! разворуют как всегда! уже трещинами пош л! тупые китазы не могут ничего нормально сделать!\n
8                                                                                                                ебать тебя разносит, шизик.\n
9                                                                                                                    обосрался, сиди обтекай\n
10                                                                       зачем ты пишешь хуйню, дегенерат? поцелуй в губы ! поцелую в засос.\n
Name: comment, dtype: object


In [297]:
#remove all links
def remove_links(column):
    return re.sub('((www.[^s]+)|(https?://[^s]+))', ' ',column)

In [298]:
toxic_dataset = toxic_dataset.apply(lambda column: remove_links(column))

In [299]:
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

In [300]:
print(russian_stopwords)
len(russian_stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

151

In [301]:
def prep(column, threshold = 1):
    threshold = threshold
    tokens = mystem.lemmatize(column.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
             and token !=" " \
             and token.strip() not in punctuation]
    column = " ".join([word for word in tokens if len(word) > threshold])
    return re.sub('[0-9]+', '', column)

In [302]:
toxic_dataset = toxic_dataset.apply(lambda column: prep(column))

In [303]:
print(toxic_dataset[5])

стан являться эталон современный система здравоохранение рф зимбабве тупой хохол


In [173]:
#toxic_comments['toxic'].value_counts()

0.0    9586
1.0    4826
Name: toxic, dtype: int64

In [255]:
#np.sum(toxic_comments.isnull().any(axis=1))

0

In [179]:
#toxic_dataset['comment'] = toxic_dataset['comment'].apply(lambda column: numbers(column))

In [304]:
#TF-IDF verctorarization
from sklearn.feature_extraction.text import TfidfVectorizer 

In [305]:
vectorizer = TfidfVectorizer(
ngram_range = (1,3),
use_idf = True,
smooth_idf = True,
norm = 'l2',
decode_error = 'replace',
max_features = 10000,
max_df = .9,
min_df = 0
)

In [306]:
X = vectorizer.fit_transform(toxic_dataset).toarray()
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [307]:
np.count_nonzero(X)

124630

In [308]:
pos_map = {v: k for k, v in enumerate(vectorizer.get_feature_names_out())}
idf_vals = vectorizer.idf_
idf_dict = {k: idf_vals[k] for k in pos_map.values()}

In [157]:
#X = vectorizer.fit_transform(toxic_comments).toarray()

In [186]:
X.T.shape

(26570, 9600)

In [385]:
#Create a list of tokens without stemming
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()
list_of_tok_tweets = []
for f in toxic_dataset:
    tweets = tweet_tokenizer.tokenize(f)
    list_of_tok_tweets.append(tweets)

In [397]:
import spacy
from spacy import displacy
from collections import Counter

In [399]:
nlp = spacy.load('ru_core_news_sm')

In [402]:
#Create Pos tags and save it as a string
list_of_tweet_tags = []
for f in list_of_tok_tweets:
    tags = nlp(f)
    list_of_tweet_tags.append(tags)

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'list'>